In [1]:
import sys
import json
import time
import os
import pandas as pd
import numpy as np
import torch
from tqdm import tqdm
import csv
import time
from functools import partial
sys.path.append('/home/evorsin/DS/pqt_loader/sequence_dataloader/')

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2
# функционал подготовки даталоадера
from webdataset_fork_0162.dataloader import *

#функционал тренировки и валидации
from examples.mts_23.engine import train_fn, val_fn, inf_fn, EarlyStop
#класс для параметров пайплайна
from utils.functions import Args, base_conf, load_pickle,seed_everything
#класс для модели
from examples.mts_23.model import RNNClsReceipts

In [4]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

# Загрузка параметров и вспомогтаельных справочников

In [5]:
url2kmeans = load_pickle('/home/evorsin/DS/mts_23/data/urls/url2kmeans.bin')

In [6]:
url2url = {k:k for k in url2kmeans}

In [7]:
params = Args(**base_conf)

In [8]:
params.cont_features=[]
params.cont_features_const=['price_sc','diff_date_m']

#params.cont_features = [col+'_sc' for col in params.cont_features]
#params.cont_features= params.cont_features+['days_from_first_buy_log1p']

params.emb_params =  {

  'part_of_day_enc': {'vocabulary_size': 5, 'dim': 2},
  'request_cnt': {'vocabulary_size': 16, 'dim': 4},
  'region_name_enc': {'vocabulary_size': 82, 'dim': 8},
  'city_name_enc': {'vocabulary_size': 986, 'dim': 4},
  'date_dow':{'vocabulary_size': 8, 'dim': 4},
  'date_day':{'vocabulary_size': 32, 'dim': 4},
  'date_month':{'vocabulary_size': 13, 'dim': 2},
  #'url_host_enc':{'vocabulary_size': 51, 'dim': 4},
   'url_host_enc':{'vocabulary_size': 200000, 'dim': 512}
                     }
params.emb_params_const =  {
  'cpe_manufacturer_name_enc': {'vocabulary_size': 38, 'dim': 4},
  'cpe_model_name_enc': {'vocabulary_size': 600, 'dim': 4},
  'cpe_type_cd_enc': {'vocabulary_size': 5, 'dim': 2},
  'cpe_model_os_type_enc': {'vocabulary_size': 4, 'dim': 2},
  'price_nan': {'vocabulary_size': 3, 'dim': 2},
                     }
params.target_col_bin ='is_male'
params.target_col_multyclass='age'
params.seq_len =512

In [9]:
mapper_conf = {'url_host_enc':url2kmeans, 'nan_v':18}

In [10]:
mapper_conf = {'url_host_enc':url2url, 'nan_v':1}

In [9]:
#!pip install webdataset==0.1.62

# debug dataloader

In [74]:
def get_dataloader_(
    path_to_data,
    params,
    batch_size=None,
    num_workers=None,
    fasttext_emb=None,
    shuffle=False,
    with_target=True,
    resample_n=None,
    mapper_conf=None
):
    if batch_size is None:
        batch_size = params.batch_size
    if num_workers is None:
        num_workers = params.num_workers
    
    const_feats = ["user_id", "user_tr_count"]+list(params.emb_params_const.keys())\
    + params.cont_features_const 
    
    if with_target:
        const_feats =const_feats+ ['is_male', 'age']

    standart_processors = (
        FeatureProcessorConst(const_feats),
        FeatureProcessorCallable(["price_nan"], lambda x:x+1),
        FeatureProcessorCallable(["url_host_enc"], lambda x:mapper_conf['url_host_enc'].get(x,18)+1),
        FeatureProcessorCallable(["age"], lambda x:x-1),
    )

    test_feature_processor = FeatureProcessorCombiner(
        [*standart_processors], max_seq_len=params.seq_len
    )

    feature_names = ["url_host_enc"]\
            +test_feature_processor.additional_column_names\
            + params.cont_features\
            + params.cont_features_const\
            + list(params.emb_params.keys())\
            + list(params.emb_params_const.keys())
    feature_names = list(set(feature_names))

    #fasttext_cols = [f"fasttext_{i}" for i in range(256)]
    #feature_names = [col for col in feature_names if col not in fasttext_cols]
    #cont_feats = [col for col in params.cont_features if col not in fasttext_cols]
    cont_feats = params.cont_features + params.cont_features_const
    resample_col = params.__dict__.get("target_col", "target")


    collator_down = SequenceCollator(
        feature_names=feature_names,
        continious_features=cont_feats,
        count_len_by="url_host_enc",
        resample_n=resample_n,
        resample_col=resample_col,
        fasttext_emb=fasttext_emb,
    )

    test_down = get_dataset(
        path_to_data,
        test_feature_processor,
        collator_down,
        batch_size,
        shuffle,
        hdfs=False,
    )

    test_down_loader = inf_loop(
        partial(prepare_loader, test_down, num_workers=num_workers)
    )

    return test_down_loader

In [48]:
df = pd.read_parquet('/home/evorsin/DS/mts_23/data/CV/fold_1/fold_1_part_0.parquet')

In [15]:
params.cont_features=[]
params.cont_features_const=['price_sc','diff_date_m']

#params.cont_features = [col+'_sc' for col in params.cont_features]
#params.cont_features= params.cont_features+['days_from_first_buy_log1p']

params.emb_params =  {
  #'cpe_manufacturer_name_enc': {'vocabulary_size': 37, 'dim': 4},
  #'cpe_model_name_enc': {'vocabulary_size': 599, 'dim': 64},
  #'cpe_type_cd_enc': {'vocabulary_size': 4, 'dim': 2},
  #'cpe_model_os_type_enc': {'vocabulary_size': 3, 'dim': 2},
  'part_of_day_enc': {'vocabulary_size': 5, 'dim': 2},
  'request_cnt': {'vocabulary_size': 16, 'dim': 4},
  'region_name_enc': {'vocabulary_size': 82, 'dim': 8},
  'city_name_enc': {'vocabulary_size': 986, 'dim': 4},
  'date_dow':{'vocabulary_size': 8, 'dim': 4},
  'date_day':{'vocabulary_size': 32, 'dim': 4},
  'date_month':{'vocabulary_size': 13, 'dim': 2},
  #'url_host_enc':{'vocabulary_size': 51, 'dim': 16},
    'url_host_enc':{'vocabulary_size': 200000, 'dim': 512}
                     }
params.emb_params_const =  {
  'cpe_manufacturer_name_enc': {'vocabulary_size': 38, 'dim': 4},
  'cpe_model_name_enc': {'vocabulary_size': 600, 'dim': 64},
  'cpe_type_cd_enc': {'vocabulary_size': 5, 'dim': 2},
  'cpe_model_os_type_enc': {'vocabulary_size': 4, 'dim': 2},
  'price_nan': {'vocabulary_size': 3, 'dim': 2},
                     }
params.target_col_bin ='is_male'
params.target_col_multyclass='age'

In [16]:
train_path = '/home/evorsin/DS/mts_23/data/CV/fold_1'

In [28]:
params.seq_len=512
#params.target_col = 'is_male'

In [29]:
train_loader = get_dataloader(train_path,
                                      params,
                                      batch_size=16,
                              num_workers=0,
                                      fasttext_emb=None, shuffle=False, resample_n=None,mapper_conf=mapper_conf)

In [30]:
d= next(iter(train_loader))

In [32]:
d

{'lengths': [512,
  512,
  280,
  145,
  512,
  512,
  115,
  512,
  203,
  97,
  512,
  512,
  512,
  512,
  27,
  244],
 'url_host_enc': tensor([[  7,   4,   3,  ..., 103,  16,   7],
         [ 51,  88,   4,  ...,  19, 316,  38],
         [ 11, 127,  15,  ...,   0,   0,   0],
         ...,
         [  7, 145,  89,  ...,  13, 116, 159],
         [  5,   8,  14,  ...,   0,   0,   0],
         [ 23,  13,  21,  ...,   0,   0,   0]]),
 'is_male': tensor([[1.],
         [0.],
         [0.],
         [0.],
         [1.],
         [0.],
         [1.],
         [0.],
         [0.],
         [0.],
         [1.],
         [1.],
         [0.],
         [0.],
         [0.],
         [1.]], dtype=torch.float64),
 'user_tr_count': tensor([[1590],
         [1286],
         [ 280],
         [ 145],
         [1490],
         [1113],
         [ 115],
         [ 876],
         [ 203],
         [  97],
         [2083],
         [ 802],
         [2140],
         [2079],
         [  27],
         [ 244]]),

In [31]:
d['lengths']

[512, 512, 280, 145, 512, 512, 115, 512, 203, 97, 512, 512, 512, 512, 27, 244]

In [16]:
d.keys()

dict_keys(['lengths', 'user_id', 'part_of_day_enc', 'date_dow', 'region_name_enc', 'date_day', 'request_cnt', 'date_month', 'price_sc', 'cpe_model_os_type_enc', 'user_tr_count', 'cpe_manufacturer_name_enc', 'age', 'diff_date_m', 'price_nan', 'url_host_enc', 'city_name_enc', 'cpe_type_cd_enc', 'cpe_model_name_enc', 'is_male', 'mask'])

In [13]:
d['age']

tensor([[3.],
        [1.],
        [1.],
        [2.],
        [0.],
        [0.],
        [1.],
        [0.],
        [3.],
        [5.],
        [1.],
        [2.],
        [1.],
        [0.],
        [2.],
        [1.]], dtype=torch.float64)

# DEBUG model

In [17]:
params.__dict__

{'emb_params': {'part_of_day_enc': {'vocabulary_size': 5, 'dim': 2},
  'request_cnt': {'vocabulary_size': 16, 'dim': 4},
  'region_name_enc': {'vocabulary_size': 82, 'dim': 8},
  'city_name_enc': {'vocabulary_size': 986, 'dim': 4},
  'date_dow': {'vocabulary_size': 8, 'dim': 4},
  'date_day': {'vocabulary_size': 32, 'dim': 4},
  'date_month': {'vocabulary_size': 13, 'dim': 2},
  'url_host_enc': {'vocabulary_size': 51, 'dim': 16}},
 'cont_features': [],
 'rnn': {'type': 'GRU',
  'hidden_size': 128,
  'rnn_num_layers': 2,
  'rnn_bias': True,
  'bidirectional': True},
 'ffn': {'hid_linear': 128},
 'seq_len': 10,
 'batch_size': 128,
 'num_workers': 0,
 'device': 'cuda:0',
 'lr': 0.001,
 'weight_decay': 0.01,
 'l1_weight': 0,
 'step_size': 2,
 'gamma': 0.5,
 'fp16': False,
 'clip_value': 1.5,
 'batch_accum': 1,
 'smooth_factor': None,
 'early_stop': {'more_best': True, 'patience': 5},
 'cont_features_const': ['price_sc', 'diff_date_m'],
 'emb_params_const': {'cpe_manufacturer_name_enc': {'v

In [18]:
%%time
#rnn_types=['GRU', 'LSTM']
#osses=['focal','bce']
experiments = {f'exp_{i}':dict() for i in [4]}
rnn_params = {'type':'GRU',
             'hidden_size':128,
              'rnn_num_layers':2,
              'rnn_bias':False,
              'bidirectional':True
             }
ffn_params ={
    'hid_linear':128
}
params.save_checkpoints= False
params.early_stop={
    'more_best':True,
    'patience':5
}
params.rnn =rnn_params
params.ffn =ffn_params

params.dropout=0
#params.weight_decay = 0.00006

#params.device='cuda:0'
params.device='cpu'
params.smooth_factor=None
params.batch_accum =1
params.pos_weight=1
params.loss_choose ='focal'
params.fp16= False
params.seed =3111
params.spatial_dropout =0.1

CPU times: user 8 µs, sys: 5 µs, total: 13 µs
Wall time: 14.8 µs


In [19]:
model = RNNClsReceipts(params)

In [20]:
#y = batch[target_col].flatten().float().to(device, non_blocking=True)

#sample_weight = batch.get("sample_weight")
#if sample_weight is None:
#    sample_weight = torch.ones_like(y)
#sample_weight = sample_weight.flatten().to(device, non_blocking=True)
batch=d
#for feat in params.cont_features:
    #batch[feat] = batch[feat].to(device, non_blocking=True)
    #batch[feat]=
#if params.__dict__.get("fasttesx_embeddings", False):
#    batch["fasttesx_embeddings"] = batch["fasttesx_embeddings"].to(
#        device, non_blocking=True
#    )
for feat in list(params.emb_params.keys())+list(params.emb_params_const.keys()):
    batch[feat] = batch[feat].long()
#if batch.get("mask") is not None:
#    batch["mask"] = batch["mask"].to(device, non_blocking=True)

In [21]:
o1,o2 =model(batch)

In [22]:
y_bi = batch[params.target_col_bin].flatten().long()
y_mc = batch[params.target_col_multyclass].flatten().long()

In [23]:
y_mc

tensor([3, 1, 1, 2, 0, 0, 1, 0, 3, 5, 1, 2, 1, 0, 2, 1])

In [24]:
crit_mc = torch.nn.CrossEntropyLoss()
crit_bi =torch.nn.BCEWithLogitsLoss()

In [26]:
o1

tensor([[ 0.0838],
        [ 0.0985],
        [ 0.1515],
        [ 0.1452],
        [ 0.1681],
        [ 0.0655],
        [ 0.0139],
        [ 0.1829],
        [ 0.0647],
        [-0.0529],
        [ 0.1944],
        [ 0.1217],
        [-0.0226],
        [ 0.0890],
        [ 0.1705],
        [-0.0067]], grad_fn=<AddmmBackward0>)

In [28]:
crit_bi(o1.squeeze(), y_bi.float())

tensor(0.7048, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

In [85]:
crit_mc(o2, y_mc)

tensor(1.8333, grad_fn=<NllLossBackward0>)

In [30]:
sm =torch.nn.Softmax(dim=1)
sd = torch.nn.Sigmoid()

In [35]:
from sklearn.metrics import f1_score, roc_auc_score


def get_metrics(true_sex, pred_sex, true_age, pred_age):
    sex_gini = (2*roc_auc_score(true_sex, pred_sex))-1
    age_f1 = f1_score(true_age, pred_age, average='weighted')
    val_score = 2*age_f1+sex_gini
    return sex_gini, age_f1, val_score

In [46]:
(params.device!="cpu")

False

In [38]:
get_metrics(y_bi, sd(o1).squeeze().tolist(), y_mc, sm(o2).argmax(dim=1).tolist())

(0.06666666666666665, 0.09375, 0.25416666666666665)

In [36]:
roc_auc_score(y_bi,sd(o1).squeeze().tolist())

0.5333333333333333

In [43]:
sd(torch.tensor([-]))

tensor([0.1192])

In [102]:
sm(o1).tolist()

[[0.49170801043510437, 0.5082920789718628],
 [0.5065975785255432, 0.49340251088142395],
 [0.5291735529899597, 0.47082653641700745],
 [0.5006413459777832, 0.4993586540222168],
 [0.4736834168434143, 0.5263165831565857],
 [0.5002886652946472, 0.4997113049030304],
 [0.5653490424156189, 0.43465104699134827],
 [0.5458074808120728, 0.45419248938560486],
 [0.4708710014820099, 0.529128909111023],
 [0.4589308798313141, 0.5410690307617188],
 [0.48330774903297424, 0.5166922211647034],
 [0.45466673374176025, 0.5453333258628845],
 [0.5558969974517822, 0.4441030025482178],
 [0.5131338238716125, 0.48686620593070984],
 [0.5540385842323303, 0.44596147537231445],
 [0.5158259272575378, 0.48417410254478455]]

In [97]:
sm(o2).argmax(dim=1)

tensor([3, 5, 5, 4, 3, 5, 4, 5, 5, 5, 3, 5, 5, 5, 4, 5])

In [ ]:
d['price_sc'].un

In [54]:
torch.cat([k(d['price_nan'].squeeze()),k(d['price_nan'].squeeze()),d['price_sc']],dim=-1)

tensor([[ 0.2201,  0.2967,  0.2201,  0.2967, -0.6951],
        [-0.1299,  2.1604, -0.1299,  2.1604,  0.6288],
        [-0.1299,  2.1604, -0.1299,  2.1604, -0.6168],
        [-0.1299,  2.1604, -0.1299,  2.1604, -0.0754],
        [-0.1299,  2.1604, -0.1299,  2.1604,  0.3542],
        [-0.1299,  2.1604, -0.1299,  2.1604,  0.4503],
        [-0.1299,  2.1604, -0.1299,  2.1604,  2.0274],
        [-0.1299,  2.1604, -0.1299,  2.1604,  3.0389],
        [-0.1299,  2.1604, -0.1299,  2.1604, -0.7308],
        [-0.1299,  2.1604, -0.1299,  2.1604, -0.8860],
        [-0.1299,  2.1604, -0.1299,  2.1604, -0.0368],
        [-0.1299,  2.1604, -0.1299,  2.1604, -0.5000],
        [-0.1299,  2.1604, -0.1299,  2.1604,  1.3461],
        [-0.1299,  2.1604, -0.1299,  2.1604,  0.4846],
        [-0.1299,  2.1604, -0.1299,  2.1604, -0.5772],
        [-0.1299,  2.1604, -0.1299,  2.1604,  0.8707]], grad_fn=<CatBackward0>)

# DEBUG ENGINE

In [76]:
params.seq_len=700

In [100]:
train_path = '/home/evorsin/DS/mts_23/data/CV/debug/'
train_loader = get_dataloader(train_path,
                                      params,
                                      batch_size=256,
                              num_workers=2,
                                      fasttext_emb=None, 
                              shuffle=False, resample_n=None, mapper_conf=mapper_conf)
val_loader = get_dataloader(train_path,
                                      params,
                                      batch_size=256,
                              num_workers=2,
                                      fasttext_emb=None, shuffle=False, resample_n=None,
                            mapper_conf=mapper_conf)

In [101]:
ch = next(iter(val_loader))

In [66]:
del params.gamma

In [60]:
params.__dict__

{'emb_params': {'part_of_day_enc': {'vocabulary_size': 5, 'dim': 2},
  'request_cnt': {'vocabulary_size': 16, 'dim': 4},
  'region_name_enc': {'vocabulary_size': 82, 'dim': 8},
  'city_name_enc': {'vocabulary_size': 986, 'dim': 4},
  'date_dow': {'vocabulary_size': 8, 'dim': 4},
  'date_day': {'vocabulary_size': 32, 'dim': 4},
  'date_month': {'vocabulary_size': 13, 'dim': 2},
  'url_host_enc': {'vocabulary_size': 51, 'dim': 16}},
 'cont_features': [],
 'rnn': {'type': 'GRU',
  'hidden_size': 128,
  'rnn_num_layers': 2,
  'rnn_bias': False,
  'bidirectional': True},
 'ffn': {'hid_linear': 128},
 'seq_len': 700,
 'batch_size': 128,
 'num_workers': 0,
 'device': 'cpu',
 'lr': 0.001,
 'weight_decay': 0.01,
 'l1_weight': 0,
 'step_size': 2,
 'gamma': 0.5,
 'fp16': False,
 'clip_value': 1.5,
 'batch_accum': 1,
 'smooth_factor': None,
 'early_stop': {'more_best': True, 'patience': 5},
 'cont_features_const': ['price_sc', 'diff_date_m'],
 'emb_params_const': {'cpe_manufacturer_name_enc': {'vo

In [102]:
%%time
#rnn_types=['GRU', 'LSTM']
#osses=['focal','bce']
experiments = {f'exp_{i}':dict() for i in [4]}
rnn_params = {'type':'GRU',
             'hidden_size':128,
              'rnn_num_layers':2,
              'rnn_bias':False,
              'bidirectional':True
             }
ffn_params ={
    'hid_linear':128
}
params.save_checkpoints= False
params.early_stop={
    'more_best':True,
    'patience':5
}
params.rnn =rnn_params
params.ffn =ffn_params

params.dropout=0
#params.weight_decay = 0.00006

params.device='cuda:0'
#params.device='cpu'
device = torch.device(params.device)
params.batch_accum =1
params.clip_value =2
params.fp16= False
params.seed =3111
params.spatial_dropout =0.1

model = RNNClsReceipts(params)
model.to(device)

params.lr = 0.0001
params.weight_decay=0.01
optimizer = torch.optim.AdamW(model.parameters(),
                              lr=params.lr, weight_decay=params.weight_decay)
scaler = torch.cuda.amp.GradScaler()

for epoch in range(1, 10):
    loss, loss_bi, loss_mc = train_fn(model=model, dataloader=train_loader,
                          optimizer=optimizer,scaler=scaler, 
                                  params=params,device=device, total=420, scheduler=None)
    curr_lr = optimizer.param_groups[0]['lr']
    print(f'Epoch {epoch} - loss: {loss}, loss_bi: {loss_bi}, loss_mc: {loss_mc}, curr_LR: {curr_lr}')
    
    score, gini, f1  = val_fn(model=model, dataloader=val_loader, params=params,device=device, total=105)
    print(f'Epoch {epoch} - score: {score}, gini: {gini}, f1: {f1}')       

 10%|████▏                                     | 42/420 [00:10<01:37,  3.86it/s]


Epoch 1 - loss: 2.3865304787953696, loss_bi: 0.6915637368247622, loss_mc: 1.69496674764724, curr_LR: 0.0001


 40%|████████████████▊                         | 42/105 [00:04<00:06,  9.28it/s]


Epoch 1 - score: 0.632362978159601, gini: 0.13891311069882484, f1: 0.24672493373038806


 10%|████▏                                     | 42/420 [00:10<01:34,  3.99it/s]


...

Epoch 8 - score: 1.0202581801598407, gini: 0.32213246609425994, f1: 0.3490628570327904


 10%|████▏                                     | 42/420 [00:10<01:35,  3.96it/s]


Epoch 9 - loss: 2.075678059032985, loss_bi: 0.6630839833191463, loss_mc: 1.4125940742946805, curr_LR: 0.0001


 40%|████████████████▊                         | 42/105 [00:04<00:06, 10.01it/s]

Epoch 9 - score: 1.0665060098453951, gini: 0.34391027023917387, f1: 0.36129786980311057
CPU times: user 1min 31s, sys: 22 s, total: 1min 53s
Wall time: 2min 13s


# Тренировка

In [11]:
train_path = '/home/evorsin/DS/mts_23/data/CV/train/fold_1'
val_path='/home/evorsin/DS/mts_23/data/CV/fold_1'
train_loader = get_dataloader(train_path,
                                      params,
                                      batch_size=128,
                              num_workers=4,
                                      fasttext_emb=None, 
                              shuffle=True, resample_n=None, mapper_conf=mapper_conf)
val_loader = get_dataloader(val_path,
                                      params,
                                      batch_size=128,
                              num_workers=2,
                                      fasttext_emb=None, shuffle=False, resample_n=None,
                            mapper_conf=mapper_conf)

In [12]:
import torch_optimizer as optim

In [13]:
%%time
#rnn_types=['GRU', 'LSTM']
#osses=['focal','bce']
experiments = {f'exp_{i}':dict() for i in [4]}
rnn_params = {'type':'LSTM',
             'hidden_size':128,
              'rnn_num_layers':2,
              'rnn_bias':True,
              'bidirectional':True
             }
ffn_params ={
    'hid_linear':128
}
params.save_checkpoints= False
params.early_stop={
    'more_best':True,
    'patience':5
}
params.rnn =rnn_params
params.ffn =ffn_params

params.dropout=0.2
#params.weight_decay = 0.00006

params.device='cuda:0'
#params.device='cpu'
device = torch.device(params.device)
params.batch_accum =1
params.clip_value =2
params.fp16= False
params.seed =3111
params.spatial_dropout =0.2



for i, exp_name in enumerate(['v2_fold1']):
    params.describe = """just test fold2"""
    params.lr = 0.001
    params.weight_decay=0
    params.step_size=2
    params.gamma = 0.1
    epochs = 50
    path_to_save_checkpoints =f'/home/evorsin/DS/mts_23/nn_exps/rnn/{exp_name}/'
    os.system(f'mkdir -p {path_to_save_checkpoints}')
    
    early_stop = EarlyStop(path_to_save_checkpoints, params)
    seed_everything(params.seed)
    
    model = RNNClsReceipts(params)
    model.to(device)
    
    #optimizer = torch.optim.AdamW(model.parameters(),
    #                              lr=params.lr, weight_decay=params.weight_decay)
    optimizer = optim.Lamb(
    model.parameters(),
    lr= params.lr,
    betas=(0.9, 0.999),
    eps=1e-8,
    weight_decay=params.weight_decay,
)
    scaler = torch.cuda.amp.GradScaler()
    scheduler= torch.optim.lr_scheduler.StepLR(optimizer, step_size=params.step_size, gamma=params.gamma)
    with open(path_to_save_checkpoints+'lrs.csv', 'w') as csvFile:
        fieldnames =['lr', 'epoch','loss', 'loss_bi', 'loss_mc','score', 'gini', 'f1' ]
        writer = csv.DictWriter(csvFile, fieldnames=fieldnames)
        writer.writeheader()
        
        for epoch in range(1, epochs+1,1):
            print(f'Epoch {epoch} - start <<<<<<<<<<<<<<<>>>>>>>>>>>>>>>>>>>>>>>')
            time.sleep(2)
            loss, loss_bi, loss_mc = train_fn(model=model, dataloader=train_loader,
                                  optimizer=optimizer,scaler=scaler, 
                                          params=params,device=device, total=412, scheduler=None)
            curr_lr = optimizer.param_groups[0]['lr']
            print(f'Epoch {epoch} - loss: {loss}, loss_bi: {loss_bi}, loss_mc: {loss_mc}, curr_LR: {curr_lr}')
            time.sleep(2)
            score, gini, f1  = val_fn(model=model, dataloader=val_loader, params=params,device=device, total=103)
            print(f'Epoch {epoch} - score: {score}, gini: {gini}, f1: {f1}')
            writer.writerow({'lr': curr_lr, 'epoch': epoch, 'loss':loss, 'loss_bi':loss_bi,
                            'loss_mc': loss_mc, 'score': score, 'gini':gini, 'f1':f1,})
            csvFile.flush()
            es = early_stop(model, score, epoch, verbose=True )
            #scheduler.step(val_score)
            #lrfn(epoch,scheduler)
            #scheduler.step()
            if es=='STOP':
                print('____EARLY STOPPING____')
                break

Epoch 1 - start <<<<<<<<<<<<<<<>>>>>>>>>>>>>>>>>>>>>>>


1649it [03:26,  8.00it/s]                                                       


Epoch 1 - loss: 1.9944160115436904, loss_bi: 0.6021539453234074, loss_mc: 1.3922620659672613, curr_LR: 0.001


412it [00:22, 18.71it/s]                                                        


Epoch 1 - score: 1.4285111395707393, gini: 0.6031766347228305, f1: 0.4126672524239544
----------- EPOCH 1 - NEW BEST METRIC: 1.4285 ----------------
Epoch 2 - start <<<<<<<<<<<<<<<>>>>>>>>>>>>>>>>>>>>>>>


591it [01:14,  7.94it/s]                                                        


KeyboardInterrupt: 

In [11]:
from copy import deepcopy

In [14]:
ch_l={}
for i, batch in tqdm(enumerate(train_loader), total=420):
    if i>127:
        ch_l[f'b_{i}']=deepcopy(batch)
    if i==130:
        break

 31%|████████████▋                            | 130/420 [00:47<01:45,  2.75it/s]


In [15]:
ch_l.keys()

dict_keys(['b_128', 'b_129', 'b_130'])

In [18]:
cols = list(ch_l['b_128'].keys())
cols

['lengths',
 'diff_date_m',
 'url_host_enc',
 'user_tr_count',
 'price_sc',
 'age',
 'date_day',
 'cpe_type_cd_enc',
 'cpe_manufacturer_name_enc',
 'date_dow',
 'price_nan',
 'is_male',
 'user_id',
 'cpe_model_name_enc',
 'city_name_enc',
 'region_name_enc',
 'request_cnt',
 'cpe_model_os_type_enc',
 'date_month',
 'part_of_day_enc',
 'mask']

In [26]:
for col in cols:
    print(col,type(ch_l['b_128'][col]))

lengths <class 'list'>
diff_date_m <class 'torch.Tensor'>
url_host_enc <class 'torch.Tensor'>
user_tr_count <class 'torch.Tensor'>
price_sc <class 'torch.Tensor'>
age <class 'torch.Tensor'>
date_day <class 'torch.Tensor'>
cpe_type_cd_enc <class 'torch.Tensor'>
cpe_manufacturer_name_enc <class 'torch.Tensor'>
date_dow <class 'torch.Tensor'>
price_nan <class 'torch.Tensor'>
is_male <class 'torch.Tensor'>
user_id <class 'torch.Tensor'>
cpe_model_name_enc <class 'torch.Tensor'>
city_name_enc <class 'torch.Tensor'>
region_name_enc <class 'torch.Tensor'>
request_cnt <class 'torch.Tensor'>
cpe_model_os_type_enc <class 'torch.Tensor'>
date_month <class 'torch.Tensor'>
part_of_day_enc <class 'torch.Tensor'>
mask <class 'torch.Tensor'>


In [45]:
np.mean(ch_l['b_128']['lengths'])

393.44140625

In [41]:
for col in cols[1:]:
    print(col, ch_l['b_128'][col].max(), ch_l['b_129'][col].max(), ch_l['b_130'][col].max() )
    print('__')

diff_date_m tensor(5.9139) tensor(5.9139) tensor(5.9139)
__
url_host_enc tensor(50) tensor(50) tensor(50)
__
user_tr_count tensor(8898) tensor(7885) tensor(9061)
__
price_sc tensor(4.1020) tensor(3.2091) tensor(3.2365)
__
age tensor(5., dtype=torch.float64) tensor(5., dtype=torch.float64) tensor(5., dtype=torch.float64)
__
date_day tensor(31) tensor(31) tensor(31)
__
cpe_type_cd_enc tensor(1) tensor(1) tensor(3)
__
cpe_manufacturer_name_enc tensor(29) tensor(23) tensor(20)
__
date_dow tensor(7) tensor(7) tensor(7)
__
price_nan tensor(2) tensor(2) tensor(2)
__
is_male tensor(1., dtype=torch.float64) tensor(1., dtype=torch.float64) tensor(1., dtype=torch.float64)
__
user_id tensor(208062) tensor(248556) tensor(290257)
__
cpe_model_name_enc tensor(497) tensor(581) tensor(556)
__
city_name_enc tensor(917) tensor(946) tensor(938)
__
region_name_enc tensor(79) tensor(78) tensor(74)
__
request_cnt tensor(12) tensor(16) tensor(12)
__
cpe_model_os_type_enc tensor(3) tensor(3) tensor(3)
__
date_

In [3]:
45*100/60/24

3.125